### 2.1 KD-Tree构建
#### 一. KD-Tree的构建方法
1. KD树是一种, 通过不同特征上的特征值, 把样本点构建成一个二叉树的方法. 二叉树的每个节点都是样本点的集合, 叶子节点是最终较为相似的样本点的集合, 他们拥有相似的特征值."KD"表示样本点有k个特征, "tree"表示最终构建的是一个二叉树
2. 如何构建kd-tree, 考虑以下三个问题:  
    * which dimension do we split along?    
        * widest or alternative dimension   
          (选择取值范围最大的特征作为分叉的考量特征, 或交替使用不同特征进行分叉)    
    * which value do we split at? 
        * median or center point of box ignoring data in box  
          (选择中位数,或忽略数据分布选择$\frac{range}{2}$)
    * when do we stop?
        * fewer than m points left or hits minimun width  
          (叶子节点少于m个样本点,或属性达到了最小range)  
    
#### 二. KD-tree用于查找nearest neighbor
1. 一旦构建好kd-tree后, 如果要对新的样本点进行nearest neighboor查找, 只要顺着kd-tree的节点划分条件, 逐层向下查找, 一直找到叶子节点, 该叶子节点为新样本所属的叶子节点
2. 首先从新样本所属叶子节点的其他样本开始查找最邻近, 记录下最邻近节点距离n
3. 然后回溯其他节点集合
 <img src="../../img/kdtree.png" width="65%" height="65%">
2. 对应文章检索, 上图说明, kd树的每个叶子节点都是一篇文章的向量表示.其中:  
  1. 绿色点: 带查询文章
  2. 蓝色框: 带查询文章所属的叶子节点包括的所有文章
3. kd树搜索1-最临近的方法: 
  1. 找到带检索文章$x_q$所在的叶子节点, 比较该叶子节点上的文章与带检索文章的距离,取最近的一个作为1-nn
  2. 回溯分支, 判断该分支的矩形边界中是否会有与待查文章$$x_q$$距离小于已知最临近距离$r$的文章. 方法是: 
    1. 从几何空间上看, 判断以$x_q$为球心, $r$为半径的球是否与该空间矩形边界有重叠(overlap)
    2. 实际上, 在构造树的过程中, 记录下叶子节点的属性取值范围$[min,max]$,确保在所有属性的$x^{min}和x^{max}$上,与$x_q$的距离都小于$|{ x }_{ q }^{ k }-{ x }_{ NN }^{ k }|$, 否则对该分支剪枝, 不去计算该叶子节点内的文章距离
  <img src="../../img/prunekd.png"  width="60%" height="60%">
  
#### 三. KD树的搜索复杂度
1. KD树构建的时间复杂度(N个叶子节点) : $Nlog(N)$   
2. 1-NN搜索复杂度 : $O(log(N))$到$O(N)$
3. KD树在多维空间下的效果并不好

#### 四. kd树的近似k-nn搜索
1. 设待查询节点$x_q$与所属叶子节点上最临近文档的距离为$r$. 在上述策略在回溯分支时, 会忽略存在文档距离$>r$的叶子节点.而KD-树近似搜索算法中, 要裁剪掉距离大于$\frac { r }{ \alpha  } (\alpha >1)$的叶子节点. 这样做的意义在于:  
  1. 当我们返回最近距离为$r$的最临近文档时, 保证没有距离大于$\frac { r }{ \alpha  }$的文档
  2. 由于裁剪分支的规则更严格$(\alpha >1)$, 所以会大大减少需要计算距离的文档数量(忽略整个叶子节点上的文档)

2. 为什么KD-树不适用于高维数据
  1. 当feature维度很高(d很大)时,由于每个feature都会至少包含两个分支, 所以最终构建一颗完整KD树所需要的样本点会很多($N>>2^d$). 二实际上, 往往没有这么多的样本. 而且高纬度下, 决策边界称为超立方体, 这些立方体往往都会有至少在一个属性维度上和已知最临近画出的超球体交叉, 使得剪枝效果大大下降
  2. KD-树的距离计算对无关feature很敏感. 而很多feature往往作为噪声出现, 需要另外学习哪些feature较为重要